# Cash flow management problem
Details of the problem are as follows:
+ Liability = [150.0,100.0,-200.0,200.0,-50.0,-350.0]
+ Financial Instruments
    + Line of credit: Limit 100K, interest = 1% per month
    + Commercial paper: Unlimited, duration = 3 months, interest = 2% for 3 months
    + Bank account: Unlimited, interest = 0.3% per month

In [1]:
# import relevant modules
import numpy as np
from scipy.optimize import linprog

In [2]:
liability = np.array([150.0,100.0,-200.0,200.0,-50.0,-350.0])
ntimes = len(liability);
paper_period = 3;
paper_int = 2;
rf_int = 0.3;
loan_int = 1;
loan_limit = 100;

**Linprog syntax**

Minimize: c^T * x

Subject to: A_ub * x <= b_ub A_eq * x == b_eq

**res = linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='simplex', callback=None, options=None)**

The bounds for each independent variable in the solution, which can take one of three forms:
+ None : **The default bounds, all variables are non-negative.** 
+ (lb, ub) : If a 2-element sequence is provided, the same lower bound (lb) and upper bound (ub) will be applied to all variables.
+ [(lb_0, ub_0), (lb_1, ub_1), …] : If an n x 2 sequence is provided, each variable x_i will be bounded by lb[i] and ub[i].
+ Infinite bounds are specified using -np.inf (negative) or np.inf (positive).


Variables
+ x variables = first (ntimes-1) variables
+ y variables = next (ntimes-3) variables
+ z variables = next ntimes variables

In [3]:
nvars = (ntimes-1) + (ntimes-paper_period) + ntimes
objvec = np.zeros(nvars)
objvec[nvars-1] = 1; # the last component is to be maximized

In [4]:
# Equality constraints
Aeq = np.zeros((ntimes,nvars))
for t in range(ntimes):
    # coefficients of x_t
    if (t<ntimes-1):
        Aeq[t,t] = 1; # corresponds to x_t
    if (t>0):
        Aeq[t,t-1] = -(1+loan_int/100); # corresponds to x_{t-1}
    
    # coefficients of y_t
    yoffset = ntimes-1
    if (t<ntimes-paper_period):
        Aeq[t,yoffset+t] = 1;
    if (t>paper_period-1):
        Aeq[t,yoffset+t-paper_period] = -(1+paper_int/100)
    
    # coefficient of z_t
    zoffset = (ntimes-1)+(ntimes-paper_period)
    if (t>0):
        Aeq[t,zoffset+t-1] = 1+rf_int/100
    Aeq[t,zoffset+t] = -1;

beq = liability

In [6]:
Aub = np.zeros((ntimes-1,nvars))
for t in range(ntimes-1):
    Aub[t,t] = 1
bub = loan_limit*np.ones(ntimes-1)

In [7]:
# set up and solve the optimization problem
res = linprog(-objvec,Aub,bub,Aeq,beq)

In [8]:
if (res.status == 0): # if optimization solved correctly
    var = res.x
    x = var[0:ntimes-1]
    y = var[yoffset:yoffset+(ntimes-paper_period)]
    z = var[zoffset:zoffset+ntimes]
    print("objective value = ", -res.fun)
    print("x = ", x)
    print("y = ", y)
    print("z = ", z)
else:
    print(res.message)

objective value =  142.496949153
x =  [  0.          50.98039216   0.           0.           0.        ]
y =  [ 150.           49.01960784  203.43436358]
z =  [   0.            0.          351.9441675     0.            0.
  142.49694915]
